In [1]:
import os
import pickle

from pathlib import Path

In [2]:
def get_file_info(filename):
    filename_parts = filename.split('_')
    return f'{filename_parts[0]}_{filename_parts[1]}', int(filename_parts[2][1:])

In [3]:
LOGS = Path('../logs/')
result_dict = {}

for f in os.listdir(LOGS):
    if 'test.txt' in f: 
        fid, epochs = get_file_info(f)
        result_dict[fid] = {
            'epochs' : epochs
        }

In [4]:
for f in result_dict.keys():
    # Get training data
    filename = LOGS / f'{f}_E{result_dict[f]["epochs"]}_train.txt'
    file = open(filename)
    
    train_loss = []
    train_accuracies = []
    
    result_dict[f]['oversample'] = 0
    
    for line in file:
        if 'Loss function' in line:
            loss_function = line.split(' :: ')[1].strip('\n')
        elif 'Optimizer' in line:
            optimizer = line.split(' :: ')[1].strip('\n')
        elif 'Learning rate' in line:
            lr = float(line.split(' :: ')[1].strip('\n'))
            if lr == 1e-3: lr = '1e-3'
            elif lr == 1e-4: lr = '1e-4'
            elif lr == 1e-5: lr = '1e-5'
        elif 'loss' in line:
            loss = float((line.split(': ')[1]).strip('\n'))
            train_loss.append(loss)
        elif 'current accuracy' in line:
            acc = float((line.split(': ')[2]).strip('\n'))
            train_accuracies.append(acc)
        elif 'Oversample' in line:
            result_dict[f]['oversample'] = int(line.split(' :: ')[1].strip('\n'))
            
    file.close()
            
    result_dict[f]['loss_function'] = loss_function
    result_dict[f]['optimizer'] = optimizer
    result_dict[f]['learning_rate'] = lr
    result_dict[f]['loss_values'] = train_loss
    result_dict[f]['accuracies'] = train_accuracies
    result_dict[f]['best_accuracy'] = max(train_accuracies)
    
    
    # -----------------------------------------
    # Get testing data
    filename = LOGS / f'{f}_E{result_dict[f]["epochs"]}_test.txt'
    file = open(filename)
    
    for line in file:
        if ' = ' in line:
            kpi = line.split(' = ')[1].strip('(').strip(')\n')
            kpi = kpi.split(', ')
            TP, TN, FP, FN = int(kpi[0]), int(kpi[1]), int(kpi[2]), int(kpi[3])
        elif ' : ' in line:
            if 'Accuracy' in line:
                accuracy = float(line.split(' : ')[1].strip('\n'))
            elif 'Recall' in line:
                recall = float(line.split(' : ')[1].strip('\n'))
                if recall < 0: recall = -0.1
            elif 'Precision' in line:
                precision = float(line.split(' : ')[1].strip('\n'))
                if precision < 0: precision = -0.1
            elif 'F1-score' in line:
                f1 = float(line.split(' : ')[1].strip('\n'))
                if precision < 0: f1 = 0
    file.close()
    
    result_dict[f]['TP'] = TP
    result_dict[f]['TN'] = TN
    result_dict[f]['FP'] = FP
    result_dict[f]['FN'] = FN
    
    result_dict[f]['accuracy'] = round(accuracy, 4)
    result_dict[f]['recall'] = round(recall, 4)
    result_dict[f]['precision'] = round(precision, 4)
    result_dict[f]['f1'] = round(f1, 4)

In [5]:
with open('results_summary.pickle', 'wb') as handle:
    pickle.dump(result_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)